In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


from keras.layers import Dense, Input
from keras.layers import Conv3D, Flatten, Lambda
from keras.layers import Reshape, Conv3DTranspose
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

import nibabel as nib
import numpy as np
import keras
from keras import losses

K.clear_session()


Using TensorFlow backend.
W0621 14:20:32.224380 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0621 14:20:32.226109 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0621 14:20:32.250028 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
from google.colab import drive

drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Load Data

In [0]:

# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# then z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def normalize_value(a):
    min = -2048 #np.min(numpy_data)
    max = 3000 #np.max(numpy_data)       
    return  ( a - min ) / (max - min)
  
def fileloader(ends):
  array = []
  file_dir_train = "/gdrive/My Drive/skulls_nifti/downsampled"
  for root, dirnames, filenames in os.walk(file_dir_train):
      for filename in filenames:

          if filename.endswith(ends):
              print(filename)
              #x_train = np.append(x_train, np.load(os.path.join(root,filename)))
              img = nib.load(os.path.join(root,filename)).get_data()
              img[img < 700] = -2048
              img[img > 3000] = -2048
              img = img.astype(np.float32)
              img = normalize_value(img)
              #data = img.get_fdata()
              array.append(img)
              continue
              
  return array

class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)#line
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        # We don't use this output.
        return x


In [0]:
ends_orig = '_15.nii.gz'
ends_dmg = 'original.nii.gz'

x_train = fileloader(ends_orig)
x_test = fileloader(ends_dmg)

x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
print(len(x_train))
print(len(x_test))
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
x_dmg = x_train
x_orig = x_test

annakova_15.nii.gz
ambartsumyan_15.nii.gz
antonyan_15.nii.gz
alekseeva_15.nii.gz
agabekova_15.nii.gz
aleev_15.nii.gz
aksyanova_15.nii.gz
aleksandrova_15.nii.gz
man1_15.nii.gz
baranova_15.nii.gz
annakova_original.nii.gz
ambartsumyan_original.nii.gz
antonyan_original.nii.gz
alekseeva_original.nii.gz
agabekova_original.nii.gz
aleev_original.nii.gz
aksyanova_original.nii.gz
aleksandrova_original.nii.gz
man1_original.nii.gz
baranova_original.nii.gz
10
10


In [0]:

image_size = 128
image_volume = 256
input_shape = (image_size, image_size, image_volume, 1)
batch_size = 1
kernel_size = 4
filters = 4
latent_dim = 55
epochs = 1000

# Build model

In [0]:
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
input_y = Input(shape=input_shape, name='encoder_input_labels')
x = inputs
for i in range(6):
    filters *= 2
    x = Conv3D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=(2,2,2),
               padding='same')(x)

# shape info needed to build decoder model
shape = K.int_shape(x)

# generate latent vector Q(z|X)
x = Flatten()(x)
x = Dense(50, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)



# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3] * shape[4], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3], shape[4]))(x)

for i in range(6):
    x = Conv3DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')(x)
    filters //= 2

outputs = Conv3DTranspose(filters=1,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same',
                          name='decoder_output')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
outputs =decoder(z) 
y = CustomVariationalLayer()([input_y,outputs])



W0621 14:20:35.516674 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 14:20:35.526933 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 14:20:36.295420 140007932770176 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# instantiate VAE model
vae = Model((inputs,input_y), y, name='vae')

sgd = keras.optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)
vae.compile(optimizer = sgd, loss = None)

tbCallBack = keras.callbacks.TensorBoard(log_dir='gdrive//My Drive//Graph', histogram_freq=0, write_graph=True, write_images=True)

#if args.weights:
#   vae.load_weights(args.weights)
#else:
    # train the autoencoder
print(np.array(x_orig).shape)
vae.fit(x = [x_orig,x_orig], y = None,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([x_orig,x_orig], None), callbacks=[tbCallBack])


#plot_results(models, data, batch_size=batch_size, model_name="vae_cnn")

W0621 14:20:36.367677 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(10, 128, 128, 256, 1)
Train on 10 samples, validate on 10 samples


W0621 14:20:39.825722 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0621 14:20:39.827510 140007932770176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/1000
10/10 [==============================] - 18s 2s/step - loss: 0.6779 - val_loss: 0.6493
Epoch 2/1000
10/10 [==============================] - 5s 542ms/step - loss: 0.6147 - val_loss: 0.5662
Epoch 3/1000
10/10 [==============================] - 5s 539ms/step - loss: 0.5142 - val_loss: 0.4427
Epoch 4/1000
10/10 [==============================] - 5s 542ms/step - loss: 0.3740 - val_loss: 0.2891
Epoch 5/1000
10/10 [==============================] - 5s 540ms/step - loss: 0.2294 - val_loss: 0.1678
Epoch 6/1000
10/10 [==============================] - 5s 539ms/step - loss: 0.1373 - val_loss: 0.1097
Epoch 7/1000
10/10 [==============================] - 5s 542ms/step - loss: 0.0980 - val_loss: 0.0885
Epoch 8/1000
10/10 [==============================] - 5s 545ms/step - loss: 0.0844 - val_loss: 0.0809
Epoch 9/1000
10/10 [==============================] - 5s 536ms/step - loss: 0.0795 - val_loss: 0.0780
Epoch 10/1000
10/10 [==============================] - 5s 541ms/step - loss: 0.0775 

# Сохранение модели

In [0]:
vae.save_weights("/gdrive/My Drive/model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
plot_model(encoder, to_file='encoder.png', show_shapes=True)
plot_model(decoder, to_file='decoder.png',show_shapes=True)
plot_model(vae, to_file='vae.png',show_shapes=True)

In [0]:
encoder.summary()
decoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 256 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 64, 128,  520         encoder_input[0][0]              
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 32, 32, 64, 1 8208        conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_3 (Conv3D)               (None, 16, 16, 32, 3 32800       conv3d_2[0][0]                   
__________________________________________________________________________________________________
conv3d_4 (

In [0]:
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 256 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 64, 128,  520         encoder_input[0][0]              
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 32, 32, 64, 1 8208        conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_3 (Conv3D)               (None, 16, 16, 32, 3 32800       conv3d_2[0][0]                   
__________________________________________________________________________________________________
conv3d_4 (

# Load and retrain

In [0]:
K.clear_session()

In [0]:
# instantiate VAE model
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
input_y = Input(shape=input_shape, name='encoder_input_label')
x = inputs
for i in range(6):
    filters *= 2
    x = Conv3D(filters=filters,
               kernel_size=kernel_size,
               activation='relu',
               strides=(2,2,2),
               padding='same')(x)

# shape info needed to build decoder model
shape = K.int_shape(x)

# generate latent vector Q(z|X)
x = Flatten()(x)
x = Dense(50, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)###########
z_log_var = Dense(latent_dim, name='z_log_var')(x)#####


z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3] * shape[4], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3], shape[4]))(x)

for i in range(6):
    x = Conv3DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        activation='relu',
                        strides=2,
                        padding='same')(x)
    filters //= 2

outputs = Conv3DTranspose(filters=1,
                          kernel_size=kernel_size,
                          activation='sigmoid',
                          padding='same',
                          name='decoder_output')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
outputs =decoder(z) 
y = CustomVariationalLayer()([input_y,outputs])
vae = Model((inputs,input_y), y, name='vae')
vae.load_weights("/gdrive/My Drive/model.h5")

In [0]:
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 256 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 64, 128,  520         encoder_input[0][0]              
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 32, 32, 64, 1 8208        conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_3 (Conv3D)               (None, 16, 16, 32, 3 32800       conv3d_2[0][0]                   
__________________________________________________________________________________________________
conv3d_4 (

In [0]:
idx = 0
session = K.get_session()
vae.load_weights("/gdrive/My Drive/model.h5")
for ix,layer in enumerate(vae.layers):
  if ix == 0:
    continue
  if idx>5:
    break
  if hasattr(layer, 'kernel_initializer'):
    #before = layer.get_weights()[0]
    layer.kernel.initializer.run(session=session)
    #print(before == layer.get_weights()[0])
   # evae.layers[idx].set_weights(vae.layers[idx].get_weights())

In [0]:
epochs = 1000

In [0]:

#sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
sgd = keras.optimizers.Adam(lr=0.001)
vae.compile(optimizer = sgd, loss = None)

tbCallBack = keras.callbacks.TensorBoard(log_dir='//gdrive//My Drive//Graph3', histogram_freq=0, write_graph=True, write_images=True)
mc = keras.callbacks.ModelCheckpoint('//gdrive//My Drive//Weight3//weights{epoch:08d}.h5', save_weights_only=True, period=100)

#if args.weights:
#   vae.load_weights(args.weights)
#else:
    # train the autoencoder
print(np.array(x_orig).shape)
vae.fit(x = [x_dmg,x_orig], y = None,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([x_dmg, x_orig],None), callbacks=[tbCallBack,mc])

(10, 128, 128, 256, 1)
Train on 10 samples, validate on 10 samples
Epoch 1/1000
10/10 [==============================] - 6s 646ms/step - loss: 0.0750 - val_loss: 0.0747
Epoch 2/1000
10/10 [==============================] - 6s 551ms/step - loss: 0.0747 - val_loss: 0.0745
Epoch 3/1000
10/10 [==============================] - 5s 549ms/step - loss: 0.0746 - val_loss: 0.0746
Epoch 4/1000
10/10 [==============================] - 6s 551ms/step - loss: 0.0747 - val_loss: 0.0745
Epoch 5/1000
10/10 [==============================] - 5s 549ms/step - loss: 0.0745 - val_loss: 0.0744
Epoch 6/1000
10/10 [==============================] - 6s 552ms/step - loss: 0.0744 - val_loss: 0.0731
Epoch 7/1000
10/10 [==============================] - 6s 550ms/step - loss: 0.0738 - val_loss: 0.0742
Epoch 8/1000
10/10 [==============================] - 6s 552ms/step - loss: 0.0729 - val_loss: 0.0686
Epoch 9/1000
10/10 [==============================] - 5s 548ms/step - loss: 0.0697 - val_loss: 0.0678
Epoch 10/1000
1

In [0]:
vae.save_weights("/gdrive/My Drive/model2.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
plot_model(vae, to_file='vae1.png',show_shapes=True)

In [0]:
x_predicted = vae.predict([x_dmg,x_dmg], batch_size=None)
data_path_ds = "//gdrive//My Drive//skulls_nifti//downsampled//annakova//annakova_original.nii.gz"
struct = nib.load(data_path_ds)
array = struct.get_data()
affine_arr = struct.affine


lastSkull = x_predicted
lastSkull = np.squeeze(lastSkull[0])

In [0]:
lastSkull *= 3000 + 2048
lastSkull += -2048
#print(lastSkull)
downsampled = nib.Nifti1Image(lastSkull,affine_arr)

In [0]:
nifti_img = nib.save(downsampled, '//gdrive//My Drive//14_new_reconstructed.nii.gz')

In [2]:
!git init
!git config — global user.email “konovalovBD@yandex.ru”
!git config — global user.name “BodyanBO”

Reinitialized existing Git repository in /content/.git/
usage: git config [<options>]

Config file location
    --global              use global config file
    --system              use system config file
    --local               use repository config file
    -f, --file <file>     use given config file
    --blob <blob-id>      read config from given blob object

Action
    --get                 get value: name [value-regex]
    --get-all             get all values: key [value-regex]
    --get-regexp          get values for regexp: name-regex [value-regex]
    --get-urlmatch        get value specific for the URL: section[.var] URL
    --replace-all         replace all matching variables: name value [value_regex]
    --add                 add a new variable: name value
    --unset               remove a variable: name [value-regex]
    --unset-all           remove all matches: name [value-regex]
    --rename-section      rename section: old-name new-name
    --remove-section      rem